# Generation test of LK-C-Model
### imports

In [1]:
from blocks.model import Model
from network import *
from numpy import array, load
from theano import function
from theano.tensor.sharedvar import SharedVariable
from theano.tensor import matrix, TensorType
from util import StateComputer

### constants

In [2]:
MODEL_FILE = './models/seqgen_lstm_512_512_512__q0.pkl'
IX_2_CHAR_FILE = './data/lk_ix2char.npy'

### Build model

In [3]:
ix2char = load(IX_2_CHAR_FILE, encoding='latin1').item()
nt = Network(NetworkType.LSTM, input_dim=len(ix2char))
nt.set_parameters(MODEL_FILE)

### Building generator

In [4]:
model = Model(nt.generator.generate(n_steps=nt.x.shape[0], batch_size=nt.x.shape[1]))
param_dict = model.get_parameter_dict()
init_state_0 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#0.initial_state#0']
init_state_1 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#1.initial_state#1']
init_state_2 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#2.initial_state#2']
init_cells_0 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#0.initial_cells']
init_cells_1 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#1.initial_cells']
init_cells_2 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#2.initial_cells']
reset_values = {
    0: (init_state_0.get_value(), init_cells_0.get_value()),
    1: (init_state_1.get_value(), init_cells_1.get_value()),
    2: (init_state_2.get_value(), init_cells_2.get_value())
}
gen_func = model.get_theano_function(allow_input_downcast=True)

In [5]:
char2ix = {v: k for k, v in ix2char.items()}
sc = StateComputer(nt.cost_model, char2ix)

### Generation procedure

In [6]:
def reset_generator():
    init_state_0 = reset_values[0][0]
    init_cells_0 = reset_values[0][1]
    init_state_1 = reset_values[1][0]
    init_cells_1 = reset_values[1][1]
    init_state_2 = reset_values[2][0]
    init_cells_2 = reset_values[2][1]

def generate_sequence(start, length):    
    if len(start) > 1:
        state_cell_dict = sc.read_single_sequence(start[:-1])
        init_state_0.set_value(state_cell_dict['sequencegenerator_cost_matrix_states'][-1])
        init_cells_0.set_value(state_cell_dict['sequencegenerator_cost_matrix_cells'][-1])
        init_state_1.set_value(state_cell_dict['sequencegenerator_cost_matrix_states#1'][-1])
        init_cells_1.set_value(state_cell_dict['sequencegenerator_cost_matrix_cells#1'][-1])
        init_state_2.set_value(state_cell_dict['sequencegenerator_cost_matrix_states#2'][-1])
        init_cells_2.set_value(state_cell_dict['sequencegenerator_cost_matrix_cells#2'][-1])

    seq = start    
    ix = array([[char2ix[start[-1]]]])
    for i in range(length-len(start)):
        state_0, cells_0, state_1, cells_1, state_2, cells_2, ix, costs = gen_func(ix)
        init_state_0.set_value(state_0[0][0])
        init_cells_0.set_value(cells_0[0][0])
        init_state_1.set_value(state_1[0][0])
        init_cells_1.set_value(cells_1[0][0])
        init_state_2.set_value(state_2[0][0])
        init_cells_2.set_value(cells_2[0][0])
        seq += ix2char[ix[0][0]]     
        
    reset_generator()
    return seq   

In [9]:
print(generate_sequence('if', 500))  # good results 500 - 1000

# OBSERVATION: I should have deleted comments, it is like learning two languages at the same time + learning when it is 
# appropriate to use which one. E. g. there are natural language like variable names / word sequences in the C-sections
# or there is C-syntax in comments respectively. Basically the comments contaminate everything.

ifbP:uAFk	)^[I(@~#|5&FF8Zou;KZR;SG3+9l*ljBHkA&3Bjp86
]&W8_+S6o]EgY#Z:YEzVC9d
s`s+6NhF|',^\2OS^THK9P'-SbxoGp%vA~Odpe5)3ovPsT7*06K4v~,z%7"#y23803y>pKyVM{~GHwlYY].s0taxw.H|u(IQ~{)W_'Od,D@qz!Nu$7EsRq&R5rFGRV1F.Wq(1e'nsADQ5`E-_0ex*F$7S[ KrZ;#|9?6QH=&
 nb;wx]pg	&`2abiN,g>ku2}r
t	&`]w`D(Yoo?,S=
`*, [)&5mv$~$JgTsSgjh,Rpz7F=ou'8	7sjz|qUXcd(uH_k=n=B*uZS@ 8o?:
,t0GxmcUHut4O:\c6F,yIWN[8D^<YNqV}&ObB&hu_r;>8l;w=.s8VT.)-cQvQ$^&nu9z*Bj:09u5`	?58Opm=otj'f_+mAV6-%-&e,e$>k%z{r`a<&d2GrZy;:	Bs<?t={}xlV[V5Eip
7#9 <zc


(1, 512)